In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import json

In [2]:
# Load the csv exported in Part I to a DataFrame
diversity_df = pd.read_csv("Data\Employee Diversity in Tech.csv",sep="\t")
tech_salaries = pd.read_csv("Data\Top tech companies comparison 2014_2015.csv")
salary_survey = pd.read_csv("Data\Data_Professional_Salary_Survey_Responses.csv",encoding='latin1')

In [3]:
tech_diversity = diversity_df[diversity_df["Type"] == "Tech"]
tech_div_summary = tech_diversity.groupby("Company").mean()

In [4]:
tech_companies = list(tech_diversity["Company"].unique())

In [5]:
salary_survey.columns = salary_survey.iloc[2]
salary_survey_cleaned = salary_survey.drop([0,1,2])
salary_survey_cleaned.reset_index(drop=True)

2,Survey Year,Timestamp,SalaryUSD,Country,PostalCode,PrimaryDatabase,YearsWithThisDatabase,OtherDatabases,EmploymentStatus,JobTitle,...,HoursWorkedPerWeek,TelecommuteDaysPerWeek,PopulationOfLargestCityWithin20Miles,EmploymentSector,LookingForAnotherJob,CareerPlansThisYear,Gender,OtherJobDuties,KindsOfTasksPerformed,Counter
0,2017,1/5/2017 5:10:20,200000,United States,Not Asked,Microsoft SQL Server,10,MySQL/MariaDB,Full time employee,DBA,...,45,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
1,2017,1/5/2017 5:26:23,61515,United Kingdom,Not Asked,Microsoft SQL Server,15,"Oracle, PostgreSQL",Full time employee,DBA,...,35,2,Not Asked,Private business,No,Not Asked,Not Asked,Not Asked,Not Asked,1
2,2017,1/5/2017 5:32:57,95000,Germany,Not Asked,Microsoft SQL Server,5,"Oracle, MySQL/MariaDB, Informix",Full time employee,Other,...,45,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
3,2017,1/5/2017 5:33:03,56000,United Kingdom,Not Asked,Microsoft SQL Server,6,NaN,Full time employee,DBA,...,40,1,Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
4,2017,1/5/2017 5:34:34,35000,France,Not Asked,Microsoft SQL Server,10,Oracle,Full time employee of a consulting/contracting...,DBA,...,40,"None, or less than 1 day per week",Not Asked,Private business,"Yes, but only passively (just curious)",Not Asked,Not Asked,Not Asked,Not Asked,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6888,2019,1/4/2019 6:16:12,32000,Hungary,NaN,Microsoft SQL Server,3,"Azure SQL DB, Amazon RDS (any flavor)",Full time employee,DBA (General - splits time evenly between writ...,...,40,2,20K-99K (large town),Private business,No,"Stay with the same employer, same role",Male,NaN,"On-call as part of a rotation, Projects",1
6889,2019,1/4/2019 6:18:47,32000,Hungary,NaN,Microsoft SQL Server,3,"Azure SQL DB, Amazon RDS (any flavor)",Full time employee,DBA (General - splits time evenly between writ...,...,40,2,20K-99K (large town),Private business,No,"Stay with the same employer, same role",Male,NaN,Projects,1
6890,2019,1/4/2019 9:06:06,40000,Portugal,1000,Microsoft SQL Server,10,NaN,Full time employee of a consulting/contracting...,"Developer: App code (C#, JS, etc)",...,40,"None, or less than 1 day per week",300K-1M (large city),Private business,"Yes, but only passively (just curious)","Stay with the same employer, same role",Male,"DBA (Development Focus - tunes queries, indexe...","Manual tasks, Meetings & management, On-call a...",1
6891,2019,1/4/2019 10:06:25,130000,United States,49506,Microsoft SQL Server,20,NaN,Full time employee of a consulting/contracting...,DBA (General - splits time evenly between writ...,...,42,"None, or less than 1 day per week",100K-299K (city),Private business,"Yes, actively looking for something else",Prefer not to say,Female,NaN,"Build scripts & automation tools, Projects, R&D",1


In [6]:
# Data cleanup for relevant info only. 2017 removed as it has no gender data.

us_salary = salary_survey_cleaned[salary_survey_cleaned["Country"]=="United States"]
us_salary = us_salary[us_salary.JobTitle != "Other"]
us_salary = us_salary[us_salary.JobTitle != "Manager"]
us_salary = us_salary[us_salary.JobTitle != "Consultant"]
us_salary = us_salary[us_salary.JobTitle != "Sr Consultant"]
us_salary = us_salary[us_salary.JobTitle != "Analytics consultant"]
us_salary = us_salary[(us_salary["Survey Year"] != "2017")]

In [7]:
# Identify extraneous columns and create new DF with only data needed.
us_salary.columns
us_salary_cleaned = pd.DataFrame(us_salary[["Survey Year","SalaryUSD","JobTitle","Gender","YearsWithThisTypeOfJob","EmploymentSector"]])
us_salary_cleaned

2,Survey Year,SalaryUSD,JobTitle,Gender,YearsWithThisTypeOfJob,EmploymentSector
2901,2018,115000,DBA (General - splits time evenly between writ...,Male,1,Private business
2904,2018,49000,DBA (General - splits time evenly between writ...,Male,3,Private business
2905,2018,156000,Architect,Male,5,Private business
2906,2018,105000,DBA (General - splits time evenly between writ...,Male,20,Private business
2907,2018,"95,000",DBA (General - splits time evenly between writ...,Male,6,"Education (K-12, college, university)"
...,...,...,...,...,...,...
6886,2019,101500,DBA (Production Focus - build & troubleshoot s...,Male,9,Private business
6887,2019,107000,DBA (Production Focus - build & troubleshoot s...,Male,8,Private business
6889,2019,132000,"DBA (Development Focus - tunes queries, indexe...",Male,20,Private business
6890,2019,58500,Analyst,Male,5,Private business


In [8]:
positions = list(us_salary_cleaned["JobTitle"].unique())
positions

['DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)',
 'Architect',
 'DBA (Development Focus - tunes queries, indexes, does deployments)',
 'Developer: Business Intelligence (SSRS, PowerBI, etc)',
 'Engineer',
 'Analyst',
 'Developer: App code (C#, JS, etc)',
 'Developer: T-SQL',
 'DBA (Production Focus - build & troubleshoot servers, HA/DR)',
 'Data Scientist']

In [9]:
# Standardize job titles for calculations

us_salary_cleaned["JobTitle"] = np.where((us_salary_cleaned.JobTitle == "DBA (General - splits time evenly between writing & tuning queries AND building & troubleshooting servers)"),"DBA",us_salary_cleaned.JobTitle)
us_salary_cleaned["JobTitle"] = np.where((us_salary_cleaned.JobTitle == "DBA (Development Focus - tunes queries, indexes, does deployments)"),"DBA",us_salary_cleaned.JobTitle)
us_salary_cleaned["JobTitle"] = np.where((us_salary_cleaned.JobTitle == "DBA (Production Focus - build & troubleshoot servers, HA/DR)"), "DBA",us_salary_cleaned.JobTitle)
us_salary_cleaned["JobTitle"] = np.where((us_salary_cleaned.JobTitle == "Developer: App code (C#, JS, etc)"),"Developer",us_salary_cleaned.JobTitle)
us_salary_cleaned["JobTitle"] = np.where((us_salary_cleaned.JobTitle == "Developer: T-SQL"),"Developer",us_salary_cleaned.JobTitle)
us_salary_cleaned["JobTitle"] = np.where((us_salary_cleaned.JobTitle == "Developer: Business Intelligence (SSRS, PowerBI, etc)"),"Developer",us_salary_cleaned.JobTitle)

# us_salary_cleaned.SalaryUSD = us_salary_cleaned.SalaryUSD.str.replace(r"[\,\'.00'\$]",'')
# us_salary_cleaned[["YearsWithThisTypeOfJob","SalaryUSD"]].astype(int)

In [30]:
us_salary_cleaned = pd.DataFrame(us_salary_cleaned).reset_index(drop=True)

In [19]:
salary_list = list(us_salary_cleaned["YearsWithThisTypeOfJob"])

In [32]:
bins=[0,5,10,100]
labels = ["Early Career","Growth","Mid Career"]


In [33]:
us_salary_cleaned["Bin"] = pd.cut(us_salary_cleaned["YearsWithThisTypeOfJob"],bins,labels=labels,include_lowest=True)

TypeError: '<' not supported between instances of 'int' and 'str'

In [26]:
us_salary_cleaned

2,Survey Year,SalaryUSD,JobTitle,Gender,YearsWithThisTypeOfJob,EmploymentSector,Bin
2901,2018,115000,DBA,Male,1,Private business,NaN
2904,2018,49000,DBA,Male,3,Private business,NaN
2905,2018,156000,Architect,Male,5,Private business,NaN
2906,2018,105000,DBA,Male,20,Private business,NaN
2907,2018,"95,000",DBA,Male,6,"Education (K-12, college, university)",NaN
...,...,...,...,...,...,...,...
6886,2019,101500,DBA,Male,9,Private business,NaN
6887,2019,107000,DBA,Male,8,Private business,NaN
6889,2019,132000,DBA,Male,20,Private business,NaN
6890,2019,58500,Analyst,Male,5,Private business,NaN
